In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
from pymongo import MongoClient

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# Import AnimalShelter class from CRUD module
from animalshelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Instantiate AnimalShelter object with the necessary parameters

username = "aacuser"
password = "passw0rd1"
host = "nv-desktop-services.apporto.com"
port = "30131"
db = "AAC"
collection = "animals"
shelter = AnimalShelter(username, password, host, port, db, collection)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
# Debug: Print the first few rows of the DataFrame to verify data retrieval
print(df.head())

# Remove the '_id' column if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)
#df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        page_size=10,  # Shows 10 rows per page
        style_table={'overflowX': 'auto'},
        row_selectable='single',  # For single row selection
        selected_rows=[0],  # Select first row by default
        filter_action='native',  # Enable filtering
        sort_action='native',  # Enable sorting
        page_action='native',  # Enable pagination
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)',
            }
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },

    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[30.75, -97.48], zoom=10, children=[
                  dl.TileLayer(id="base-layer-id"),
              dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                       children=[
                           dl.Tooltip(dff.iloc[row, 4]),
                           dl.Popup([
                               html.H1("Animal Name"),
                               html.P(dff.iloc[row, 9])
                           ])
                       ])
              ])
    ]

    

app.run_server()

   rec_num age_upon_outcome animal_id animal_type                     breed  \
0        1          3 years   A746874         Cat    Domestic Shorthair Mix   
1        3          2 years   A716330         Dog   Chihuahua Shorthair Mix   
2       10         3 months   A664290         Cat    Domestic Shorthair Mix   
3        6          5 years   A696004         Dog  Cardigan Welsh Corgi Mix   
4       12           1 year   A664843         Dog              Pit Bull Mix   

         color date_of_birth             datetime            monthyear  \
0  Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1  Brown/White    2013-11-18  2015-12-28 18:43:00  2015-12-28T18:43:00   
2       Tortie    2013-09-01  2013-12-08 14:58:00  2013-12-08T14:58:00   
3  Sable/White    2010-01-27  2015-01-28 10:39:00  2015-01-28T10:39:00   
4  Brown/White    2013-06-09  2014-08-18 17:24:00  2014-08-18T17:24:00   

       name outcome_subtype outcome_type sex_upon_outcome  location_lat  \
0    

TypeError: an integer is required (got type NoneType)